# microsoft/sbom-tool Analysis

This page is an overview of the [microsoft/sbom-tool](https://github.com/microsoft/sbom-tool) highlighting our findings from knowledge graph analysis on the previous pages.

Let's break this up by section of the generated SBOM to highlight what its good at, and what it isn't.



## Overview

This tool generates SBOMs in **SPDX-2.2 JSON format**.  In it it identifies files, packages, relationships and project metadata.  Project metadata includes project name, organization name, and project version which are all provided as command line arguments upon execution.

The sbom-tool is a command line tool that has options to either generate SBOMs, or validate SBOMs.  In our experiments we've only focused on generation. Generation requires a number of command line arguments including, package name, version, and supplier organization.

## Files

The sbom-tool generates a list of all files within a parent directory recursively. For each file it identifies the path (from the parent directory) and calculates the SHA256 and SHA1 checksums as a unique file identifier.

The sbom-tool **does not identify individual file licenses nor file authors** despite the SPDX schema providing identifiers for these.  

The list of generated files includes all files within a parent directory including files labeled in the `.gitignore`.  This means all files present in a repository **including those not for a package distribution** and **files for local python virtual environments** will be identified by the sbom-tool.


## Packages

The sbom-tool uses [microsoft/component-detection](https://github.com/microsoft/component-detection) to identify dependencies of a repository.  This includes CocoaPods, Linux, Gradle, Go, Maven, NPM, NuGet, Pip, Poetry, Ruby, and Rust.  Since most of our experiments have been with python (Pip) based projects, further descriptions will refer to that.

The sbom-tool requires either a `requirements.txt` or `setup.py` file to identify project dependencies.  In it it can detect both packages using PyPI or from github (using the `git:` identifier).  Dependencies of each package are found with PyPI where a depth first search approach is used to find all transitive dependencies. 

Our studies show the sbom-tool skips all python `stdlib` packages and does not specify the python version used in the project. 

Further in the case of non python subdependencies, the sbom-tool fails to identify these. For example, `numpy` uses C/C++ based linear algebra libraries that are not detected by the sbom-tool.

Finally, the sbom-tool does not identify license and distribution information for each package, which is readily available on PyPI.

## Relationships 

The sbom-tool also identifies some relationships (specified by the SPDX standard) in the project.  However, these relationships are limited to the type `contains` where the `root-project` `contains` `some-package`.

There is opportunity to identify many other types of relationships under the SPDX standard including but not limited to the dependency tree (direct dependencies vs transitive dependencies), which files use which packages, authors to files, etc. 